# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [3]:
X = boston_features
y = pd.DataFrame(boston.target,columns = ['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

Fit the model and apply the model to the make test set predictions

In [6]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [7]:
y_hats = model.predict(X)

Calculate the residuals and the mean squared error

In [8]:
residuals = y-y_hats
residuals
from sklearn.metrics import mean_squared_error
print("MSE =", mean_squared_error(y, y_hats))

MSE = 16.628623920551465


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [9]:
def kfolds(data, k):
    df = pd.DataFrame(data)
    subset = []
    if len(data) % k == 0:
        for i in range(0,len(data), len(data)//k):
            subset.append(data[i:i+len(data)//k]) 
    else:
        h=0
        j = len(data)%k
        while j > 0:
            subset.append(data[h:h+len(data)//k+1])
            h+=len(data)//k+1
            j-=1
        for i in range(h, len(data), len(data)//k):
            subset.append(data[i:i+len(data)//k])
    return subset 

### Apply it to the Boston Housing Data

In [10]:
# Make sure to concatenate the data again
df = pd.concat([X, y], axis=1)
df.head()
# 

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,0.542096,1.0,296.0,15.3,1.000000,-1.275260,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,0.623954,2.0,242.0,17.8,1.000000,-0.263711,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,0.623954,2.0,242.0,17.8,0.989737,-1.627858,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,0.707895,3.0,222.0,18.7,0.994276,-2.153192,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,0.707895,3.0,222.0,18.7,1.000000,-1.162114,36.2


In [11]:
folds = kfolds(df, 5)
len(folds[0])
len(folds)

5

In [12]:
X_feats = ['CRIM', 'ZN', 'INDUS','CHAS', 'NOX', 'RM', 'AGE','DIS', 'RAD', 'TAX','PTRATIO', 'B', 'LSTAT']
y_feat = 'target'

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [17]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
    train = pd.concat([fold for i,fold in enumerate(folds) if i != n])
    test = folds[n]
    # Fit a linear regression model
    model = LinearRegression()
    model.fit(train[X_feats], train[y_feat])
    y_hat_train = model.predict(train[X_feats])
    y_hat_test = model.predict(test[X_feats])
    
    #Evaluate Train and Test Errors
    train_residuals = y_hat_train - train[y_feat]
    test_residuals = y_hat_test - test[y_feat]
    train_mse = np.mean(train_residuals.apply(lambda x: x**2))
    test_mse = np.mean(test_residuals.apply(lambda x: x**2))
    train_errs.append(train_mse)
    test_errs.append(test_mse)

print("Training errors: ", train_errs)
print("Testing erros: ", test_errs)

Training errors:  [17.9185670542463, 17.3577081046629, 15.545678258525873, 11.037622389644579, 17.23404426556592]
Testing erros:  [13.016192102045757, 14.62832183142467, 24.81432997168207, 55.24107772637723, 19.02233799916951]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [31]:
from sklearn.model_selection import cross_val_score
model = LinearRegression()
model_fit = model.fit(df[X_feats], df.target)
cv_5_results = cross_val_score(model, X, y, cv=5, scoring = "neg_mean_squared_error")
cv_5_results 

array([-13.0161921 , -14.62832183, -24.81432997, -55.24107773,
       -19.022338  ])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [28]:
from sklearn.metrics import mean_squared_error
all_mse = []
for i, fold in enumerate(folds):
    y_hats = model.predict(fold[X_feats])
    mse = mean_squared_error(fold['target'], y_hats)
    all_mse.append(mse)
    print("Fold #{} = {}".format(i+1, mse))
    
print(np.mean(all_mse))

Fold #1 = 9.371372995567205
Fold #2 = 12.19727132786219
Fold #3 = 16.45990459179626
Fold #4 = 30.985476377434058
Fold #5 = 19.02233799916951
17.607272658365844


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!